In [1]:
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [39]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# All of these variables substantially change calculation time
stk_object.dt = 30
n_targets = 15
n_sats = 12
n_pop = 10
n_gen = 5

Opt = Optimizer(stk_object,n_pop,n_gen,n_sats)

c:\Users\jjfoo\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\jjfoo\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Satellite' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [8]:
# Generating Targets

# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('Input_Files/Targets_Polygon.txt')

# Writing random points within the polygon to a target file.
targets_filename = 'Input_Files/Targets_File.txt'
polygon_random_points(poly,n_targets,targets_filename)

# Loading targets into stk from file.
stk_object.Target_Loader(targets_filename)

# # Plotting the polygon and generated targets on the map.
# plot_targets_and_polygon(poly,targets_filename)

In [40]:
# Calling optimizer object and running optimization

stk_object.dt = 30
stk_object.Interpolate = True
hof,percent,times,sats = Opt.run()
Opt.cost_function(hof[0])
print(pd.DataFrame(percent))

-- Generation 0 --
------------------------------------------------------------------------
- Computing_AzEl [==========] 45/45 [100%] in 0.7s (65.32/s)                    
------------------------------------------------------------------------
- Computing_AzEl [==========] 75/75 [100%] in 0.7s (98.62/s)                    
------------------------------------------------------------------------
- Computing_AzEl [==========] 180/180 [100%] in 1.9s (89.98/s)                  
------------------------------------------------------------------------
- Computing_AzEl [==========] 120/120 [100%] in 2.9s (41.99/s)                  
------------------------------------------------------------------------
- Computing_AzEl [==========] 45/45 [100%] in 0.5s (97.17/s)                    
------------------------------------------------------------------------
- Computing_AzEl [==========] 165/165 [100%] in 3.8s (43.62/s)                   ▅▃▁ 147/165 [89%] in 3s (~0s, 43.7/s) 
         avg      

In [57]:
from requirements.OCD_1_1 import Check_Lifetime

for h in hof:
    print(h)
    Opt.cost_function(h)
    Check_Lifetime(stk_object).run()

[551, 114.06693387905179, 155, 2.489980685706235, 1.7096504479575423]
------------------------------------------------------------------------
- Computing_AzEl [==========] 30/30 [100%] in 0.8s (37.43/s)                    
Satellite_0 decay is estimated to be on 21 Nov 2025 10:57:35.191 after 6844 orbits.The lifetime is 1.2 years.
Satellite_1 decay is estimated to be on 21 Nov 2025 10:57:35.192 after 6844 orbits.The lifetime is 1.2 years.
[582.0239235695132, 72.29799544275846, 94.1719377761291, 3.590134616506697, 3.938285747848024]
------------------------------------------------------------------------
- Computing_AzEl [==========] 45/45 [100%] in 1.2s (39.63/s)                    ▃▅ 41/45 [91%] in 1s (~0s, 40.1/s) 
Satellite_0 decay is estimated to be on 16 Nov 2026 23:42:04.491 after 12277 orbits.The lifetime is 2.2 years.
Satellite_1 decay is estimated to be on 19 Nov 2026 19:20:14.425 after 12319 orbits.The lifetime is 2.2 years.
Satellite_2 decay is estimated to be on 18 Nov 202

Satellite_0 decay is estimated to be on 16 Nov 2026 23:42:04.491 after 12277 orbits.The lifetime is 2.2 years.
Satellite_1 decay is estimated to be on 19 Nov 2026 19:20:14.425 after 12319 orbits.The lifetime is 2.2 years.
Satellite_2 decay is estimated to be on 18 Nov 2026 10:55:13.969 after 12299 orbits.The lifetime is 2.2 years.


In [10]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=30, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

t_interp = time.time()

stk_object.dt = 30
stk_object.Interpolate = True
p1,t1,s1=Opt.cost_function(write=False)

interpolated_target_bins = stk_object.target_bins.copy()

t_interp = time.time()-t_interp

# t_sim = time.time()
# stk_object.dt = 2.5
# stk_object.Interpolate = False
# p2,t2,s2=Opt.cost_function(write=False)

# t_sim = time.time()-t_sim

# target_bins = stk_object.target_bins.copy()

------------------------------------------------------------------------
- Computing_AzEl [==========] 30/30 [100%] in 15.7s (1.87/s)                    [=====>    ] ▅▃▁ 15/30 [50%] in 9s (~9s, 1.7/s) 


In [28]:
fig = px.imshow(np.array([bin.T[::-1] for bin in interpolated_target_bins]),animation_frame=0,zmax=1,zmin=0, color_continuous_scale='Greys')
fig.update_layout({"title":f"Interpolated Data-(comp time={t_interp:.2f}s,p={p1:.2f}%,t={t1:.4f} days,sats={s1})",
                   "xaxis_title":"Azimuth",
                   "yaxis_title":"Elevation"})
fig.show()

# fig = px.imshow(np.array([bin.T[::-1] for bin in target_bins]),animation_frame=0)
# fig.update_layout({"title":f"Simulated Data-(comp time={t_sim:.2f}s,p={p2:.2f}%,t={t2:.4f} days,sats={s2})",
#                    "xaxis_title":"Azimuth",
#                    "yaxis_title":"Elevation"})
# fig.show()

In [38]:
import matplotlib.pyplot as plt
%matplotlib tk

N = len(stk_object.satellites)
fig,axes = plt.subplots(N//2,2)
for interval in stk_object.Intervals:
    tar = interval.target_number
    sat = interval.satellite_number
    axes[sat].plot([interval.start,interval.stop],[tar,tar],marker="|")